In [1]:
import pandas as pd
import os
import time
import numpy as np
#os.environ["CUDA_VISIBLE_DEVICES"]="6"
import tensorflow as tf
import numpy as np
import os
import random
import copy
import re
import csv
from collections import deque
from glob import glob
import bcolz
from sklearn.metrics import log_loss,auc, roc_curve
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1.0) #,gpu_options=gpu_options
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
%pwd

'C:\\Users\\chapanda\\OneDrive - Epsilon\\cp\\ACG\\03_Practise\\RnD\\crosssell'

In [22]:
path_out = 'C:\\Users\\chapanda\\data\\np\\'
#path_out = u'/userdata/projects/instakart/np/'
N = 100
debug = False
batch_size = 128
num_training_steps = 1000000
learning_rate = 0.0003
opt = 'adam'
grad_clip = 5
regularization_constant = 0.0000
warm_start_init_step = 0
early_stopping_steps = 500000
keep_prob_scalar = 1.0
enable_parameter_averaging = False
num_restarts = 0
min_steps_to_checkpoint = 50000
log_interval = 1000
loss_averaging_window = 1000
lstm_size = 300
val_multiplier = 1
WaveNet = True
dropout = 0.2

dilations=[2**i for i in range(6)]
dilations=dilations+dilations
filter_widths=[2]*6
filter_widths=filter_widths+filter_widths
skip_channels=64
residual_channels=128

prediction_dir = path_out + "predictions"
checkpoint_dir = path_out + "checkpoints"

if not os.path.isdir(prediction_dir):
    os.makedirs(prediction_dir)

if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)


In [3]:
start_time = 0
def tic():
    global start_time
    start_time = time.time()
def toc():
    global start_time
    elapsed_time = (time.time() - start_time)
    print("took me " + str(round(elapsed_time, 3))+" seconds to do this..")

In [4]:
def shape(tensor, dim=None):
    """Get tensor shape/dimension as list/int"""
    if dim is None:
        return tensor.shape.as_list()
    else:
        return tensor.shape.as_list()[dim]

In [5]:
class dataset(object):

    def __init__(self, path_out, suffix):
        
        self.filenames = [
            'user_id',
            'master_aisle_id',
            'master_department_id',
            'IsInOrder_history',
            'NextInOrder_history',
            'NumProductsFromDep_history',
            'OrderSize_history',
            'IndexInOrder_history',
            'order_number_history',
            'order_hour_of_day_history',
            'order_dow_history',
            'days_since_prior_order_history',
            'history_length',
            'ProductID1_history_a',
            'ProductID2_history_a',
            'ProductID3_history_a',
            'ProductID4_history_a',
            'ProductID5_history_a',
            'ProductID6_history_a',
            'ProductID7_history_a',
            'ProductID8_history_a',
            'ProductID9_history_a',
            'ProductID10_history_a',
            'ProductID11_history_a',
            'ProductID12_history_a',
            'ProductID13_history_a',
            'ProductID14_history_a',
            'ProductID15_history_a',
            'ProductID16_history_a',
            'ProductID17_history_a',
            'ProductID18_history_a',   
            'ProductID19_history_a',
            'ProductID20_history_a',
            'ProductID1_history',
            'ProductID2_history',
            'ProductID3_history',
            'ProductID4_history',
            'ProductID5_history',
            'ProductID6_history',
            'ProductID7_history',
            'ProductID8_history',
            'ProductID9_history',
            'ProductID10_history',
            'ProductID11_history',
            'ProductID12_history',
            'ProductID13_history',
            'ProductID14_history',
            'ProductID15_history',
            'ProductID16_history',
            'ProductID17_history',
            'ProductID18_history',   
            'ProductID19_history',
            'ProductID20_history',
            'ProductNameEmbedding_history',
            'label'
        ]
        
        self.ids = 0
        self.start = 0
        self.dataFiles = glob(path_out+'user_id*.dat') 
        self.num_chunks = len(self.dataFiles)
        self.chunks_index = 0
        
        self.start = 0 
        self.arrays = [(bcolz.open(self.dataFiles[0].replace('user_id',i))[:]) for i in self.filenames]
        self.ids = np.arange(0,len(self.arrays[0]))

    def next_batch(self, batch_size, shuffle=True, is_test=False):
        while(1):
            if (self.arrays[0].shape[0] - self.start) >= batch_size:
                batch_ids = self.ids[self.start: (self.start + batch_size)]
                self.start += batch_size
                yield {globals()[placeholder] : copy.copy(array[batch_ids]) for placeholder, array in zip(self.filenames,self.arrays)} 
            else:
                self.start = 0 
                self.chunks_index += 1
                if self.chunks_index >= self.num_chunks:
                    self.chunks_index = 0
                self.arrays = [(bcolz.open(self.dataFiles[self.chunks_index].replace('user_id',i))[:]) for i in self.filenames]
                self.ids = np.arange(0,len(self.arrays[0]))
                if shuffle:
                    random.shuffle(self.ids)

In [6]:
train = dataset(path_out,"")

In [7]:
def sequence_log_loss(y, y_hat, sequence_lengths, max_sequence_length, eps=1e-15):
    """
    Calculates average log loss on variable length sequences.

    Args:
        y: Label tensor of shape [batch size, max_sequence_length, input units].
        y_hat: Prediction tensor, same shape as y.
        sequence_lengths: Sequence lengths.  Tensor of shape [batch_size].
        max_sequence_length: maximum length of padded sequence tensor.

    Returns:
        Log loss. 0-dimensional tensor.
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.minimum(tf.maximum(y_hat, eps), 1.0 - eps)
    log_losses = y*tf.log(y_hat) + (1.0 - y)*tf.log(1.0 - y_hat)
    sequence_mask = tf.cast(tf.sequence_mask(sequence_lengths, maxlen=max_sequence_length), tf.float32)
    avg_log_loss = -tf.reduce_sum(log_losses*sequence_mask) / tf.cast(tf.reduce_sum(sequence_lengths), tf.float32)
    return avg_log_loss

In [8]:
def temporal_convolution_layer(inputs, output_units, convolution_width, causal=False, dilation_rate=[1], bias=True,
                               activation=None, dropout=None, scope='temporal-convolution-layer', reuse=False):
    """
    Convolution over the temporal axis of sequence data.

    Args:
        inputs: Tensor of shape [batch size, max sequence length, input_units].
        output_units: Output channels for convolution.
        convolution_width: Number of timesteps to use in convolution.
        causal: Output at timestep t is a function of inputs at or before timestep t.
        dilation_rate:  Dilation rate along temporal axis.

    Returns:
        Tensor of shape [batch size, max sequence length, output_units].

    """
    with tf.variable_scope(scope, reuse=reuse):
        if causal:
            shift = int((convolution_width / 2) + (int(dilation_rate[0] - 1) / 2)) #FIXED THIS
            pad = tf.zeros([tf.shape(inputs)[0], shift, inputs.shape.as_list()[2]])
            inputs = tf.concat([pad, inputs], axis=1)

        W = tf.get_variable(
            name='weights',
            initializer=tf.contrib.layers.variance_scaling_initializer(),
            shape=[convolution_width, shape(inputs, 2), output_units]
        )

        z = tf.nn.convolution(inputs, W, padding='SAME', dilation_rate=dilation_rate)
        if bias:
            b = tf.get_variable(
                name='biases',
                initializer=tf.constant_initializer(),
                shape=[output_units]
            )
            z = z + b
        z = activation(z) if activation else z
        z = tf.nn.dropout(z, dropout) if dropout is not None else z
        z = z[:, :-shift, :] if causal else z
        return z


In [9]:
def wavenet(x, dilations, filter_widths, skip_channels, residual_channels, scope='wavenet', reuse=False):
    """
    A stack of causal dilated convolutions with paramaterized residual and skip connections as described
    in the WaveNet paper (with some minor differences).

    Args:
        x: Input tensor of shape [batch size, max sequence length, input units].
        dilations: List of dilations for each layer.  len(dilations) is the number of layers
        filter_widths: List of filter widths.  Same length as dilations.
        skip_channels: Number of channels to use for skip connections.
        residual_channels: Number of channels to use for residual connections.

    Returns:
        Tensor of shape [batch size, max sequence length, len(dilations)*skip_channels].
    """
    with tf.variable_scope(scope, reuse=reuse):

        # wavenet uses 2x1 conv here
        inputs = time_distributed_dense_layer(x, residual_channels, activation=tf.nn.tanh, scope='x-proj')

        skip_outputs = []
        for i, (dilation, filter_width) in enumerate(zip(dilations, filter_widths)):
            dilated_conv = temporal_convolution_layer(
                inputs=inputs,
                output_units=2*residual_channels,
                convolution_width=filter_width,
                causal=True,# CHECK THIS OUT
                dilation_rate=[dilation],
                scope='cnn-{}'.format(i) 
            )
            conv_filter, conv_gate = tf.split(dilated_conv, 2, axis=2)
            dilated_conv = tf.nn.tanh(conv_filter)*tf.nn.sigmoid(conv_gate)

            output_units = skip_channels + residual_channels
            outputs = time_distributed_dense_layer(dilated_conv, output_units, scope='cnn-{}-proj'.format(i))
            skips, residuals = tf.split(outputs, [skip_channels, residual_channels], axis=2)

            inputs += residuals
            skip_outputs.append(skips)

        skip_outputs = tf.nn.relu(tf.concat(skip_outputs, axis=2))
        return skip_outputs


In [10]:
def lstm_layer(inputs, lengths, state_size, keep_prob=1.0, scope='lstm-layer', reuse=False, return_final_state=False):
    """
    LSTM layer.

    Args:
        inputs: Tensor of shape [batch size, max sequence length, ...].
        lengths: Tensor of shape [batch size].
        state_size: LSTM state size.
        keep_prob: 1 - p, where p is the dropout probability.

    Returns:
        Tensor of shape [batch size, max sequence length, state_size] containing the lstm
        outputs at each timestep.

    """
    with tf.variable_scope(scope, reuse=reuse):
        cell_fw = tf.contrib.rnn.DropoutWrapper(
            tf.contrib.rnn.LSTMCell(
                state_size,
                reuse=reuse
            ),
            output_keep_prob=keep_prob
        )
        outputs, output_state = tf.nn.dynamic_rnn(
            inputs=inputs,
            cell=cell_fw,
            sequence_length=lengths,
            dtype=tf.float32
        )
        if return_final_state:
            return outputs, output_state
        else:
            return outputs

In [11]:
def time_distributed_dense_layer(inputs, output_units, bias=True, activation=None, batch_norm=None,
                                 dropout=None, scope='time-distributed-dense-layer', reuse=False):
    """
    Applies a shared dense layer to each timestep of a tensor of shape [batch_size, max_seq_len, input_units]
    to produce a tensor of shape [batch_size, max_seq_len, output_units].

    Args:
        inputs: Tensor of shape [batch size, max sequence length, ...].
        output_units: Number of output units.
        activation: activation function.
        dropout: dropout keep prob.

    Returns:
        Tensor of shape [batch size, max sequence length, output_units].

    """
    with tf.variable_scope(scope, reuse=reuse):
        W = tf.get_variable(
            name='weights',
            initializer=tf.contrib.layers.variance_scaling_initializer(),
            shape=[shape(inputs, -1), output_units]
        )
        z = tf.einsum('ijk,kl->ijl', inputs, W)
        if bias:
            b = tf.get_variable(
                name='biases',
                initializer=tf.constant_initializer(),
                shape=[output_units]
            )
            z = z + b

        if batch_norm is not None:
            z = tf.layers.batch_normalization(z, training=batch_norm, reuse=reuse)

        z = activation(z) if activation else z
        z = tf.nn.dropout(z, dropout) if dropout is not None else z
        return z


In [12]:
def findAverageAUC(validation_predictions,validation_aisle,validation_ground_truths):
    
    aisles = np.unique(validation_aisle[:,0])
    cum_auc = 0.0
    cnt = 0

    for n in range(len(aisles)):

        predics_aisle = validation_predictions[np.where(validation_aisle==aisles[n])]
        actual_aisle = validation_ground_truths[np.where(validation_aisle==aisles[n])]
        
        #print(actual_aisle)
        #print(predics_aisle)
        fpr, tpr, thresholds = roc_curve(actual_aisle, predics_aisle)
        aisle_auc = auc(fpr, tpr)
        if np.isnan(aisle_auc)==False:
            cum_auc += aisle_auc
            cnt += 1
        #print(aisle_auc)
    return (cum_auc/cnt)

In [14]:
def tensor_log_loss(y, y_hat, eps=1e-15):
    """
    Calculates log loss between two tensors.

    Args:
        y: Label tensor.
        y_hat: Prediction tensor

    Returns:
        Log loss. 0-dimensional tensor.
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.minimum(tf.maximum(y_hat, eps), 1.0 - eps)
    log_loss = -tf.reduce_mean(y*tf.log(y_hat) + (1.0 - y)*tf.log(1.0 - y_hat))
    return log_loss


In [25]:
 with tf.Graph().as_default() as graph:
        
    user_id = tf.placeholder(tf.int32, [None])
    master_aisle_id = tf.placeholder(tf.int32, [None])
    master_department_id = tf.placeholder(tf.int32, [None])
    history_length = tf.placeholder(tf.int32, [None])

    IsInOrder_history = tf.placeholder(tf.int32, [None, N]) #
    order_dow_history = tf.placeholder(tf.int32, [None, N])
    order_hour_of_day_history = tf.placeholder(tf.int32, [None, N])
    days_since_prior_order_history = tf.placeholder(tf.int32, [None, N])
    OrderSize_history = tf.placeholder(tf.int32, [None, N]) #
    IndexInOrder_history = tf.placeholder(tf.int32, [None, N]) #
    order_number_history = tf.placeholder(tf.int32, [None, N])
    NumProductsFromDep_history = tf.placeholder(tf.int32, [None, N]) #
    NextInOrder_history = tf.placeholder(tf.int32, [None, N])
    
    label = tf.placeholder(tf.int32, [None])

    
    ProductID1_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID2_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID3_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID4_history_a = tf.placeholder(tf.int32, [None, N])   
    ProductID5_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID6_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID7_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID8_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID9_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID11_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID10_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID12_history_a = tf.placeholder(tf.int32, [None, N])   
    ProductID13_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID14_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID15_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID16_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID17_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID18_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID19_history_a = tf.placeholder(tf.int32, [None, N])
    ProductID20_history_a = tf.placeholder(tf.int32, [None, N])
    
    ProductNameEmbedding_history = tf.placeholder(tf.float32, [None, N, 50])

    ProductID1_history = tf.placeholder(tf.int32, [None, N])
    ProductID2_history = tf.placeholder(tf.int32, [None, N])
    ProductID3_history = tf.placeholder(tf.int32, [None, N])
    ProductID4_history = tf.placeholder(tf.int32, [None, N])   
    ProductID5_history = tf.placeholder(tf.int32, [None, N])
    ProductID6_history = tf.placeholder(tf.int32, [None, N])
    ProductID7_history = tf.placeholder(tf.int32, [None, N])
    ProductID8_history = tf.placeholder(tf.int32, [None, N])
    ProductID9_history = tf.placeholder(tf.int32, [None, N])
    ProductID11_history = tf.placeholder(tf.int32, [None, N])
    ProductID10_history = tf.placeholder(tf.int32, [None, N])
    ProductID12_history = tf.placeholder(tf.int32, [None, N])   
    ProductID13_history = tf.placeholder(tf.int32, [None, N])
    ProductID14_history = tf.placeholder(tf.int32, [None, N])
    ProductID15_history = tf.placeholder(tf.int32, [None, N])
    ProductID16_history = tf.placeholder(tf.int32, [None, N])
    ProductID17_history = tf.placeholder(tf.int32, [None, N])
    ProductID18_history = tf.placeholder(tf.int32, [None, N])
    ProductID19_history = tf.placeholder(tf.int32, [None, N])
    ProductID20_history = tf.placeholder(tf.int32, [None, N])
  
    keep_prob = tf.placeholder(tf.float32)
    is_training = tf.placeholder(tf.bool)
    
    # aisle data
    aisle_embeddings = tf.get_variable(
        name='aisle_embeddings',
        shape=[135, 50],
        dtype=tf.float32
    )
  
    # department data
    dept_embeddings = tf.get_variable(
        name='dept_embeddings',
        shape=[22, 10],
        dtype=tf.float32
    )
  
    x_aisle = tf.concat([
        tf.nn.embedding_lookup(aisle_embeddings, master_aisle_id),
        tf.nn.embedding_lookup(dept_embeddings, master_department_id),
    ], axis=1)
    x_aisle = tf.tile(tf.expand_dims(x_aisle, 1), (1, N, 1))

    # user data
    user_embeddings = tf.get_variable(
        name='user_embeddings',
        shape=[207000, min(lstm_size,3)],
        dtype=tf.float32
    )
    x_user = tf.nn.embedding_lookup(user_embeddings, user_id)
    x_user = tf.tile(tf.expand_dims(x_user, 1), (1, N, 1))

    # product data
    product_embeddings = tf.get_variable(
        name='product_embeddings',
        shape=[50000, 50],
        dtype=tf.float32
    )
       
    x_product_1a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID1_history_a))
    x_product_2a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID2_history_a))
    x_product_3a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID3_history_a))
    x_product_4a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID4_history_a))
    x_product_5a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID5_history_a))
    x_product_6a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID6_history_a))
    x_product_7a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID7_history_a))
    x_product_8a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID8_history_a))
    x_product_9a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID9_history_a))
    x_product_10a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID10_history_a))
    x_product_11a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID11_history_a))
    x_product_12a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID12_history_a))
    x_product_13a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID13_history_a))
    x_product_14a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID14_history_a))
    x_product_15a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID15_history_a))
    x_product_16a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID16_history_a))
    x_product_17a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID17_history_a))
    x_product_18a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID18_history_a))
    x_product_19a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID19_history_a))
    x_product_20a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID20_history_a))
    
    x_product_1 = tf.nn.embedding_lookup(product_embeddings, (ProductID1_history))
    x_product_2 = tf.nn.embedding_lookup(product_embeddings, (ProductID2_history))
    x_product_3 = tf.nn.embedding_lookup(product_embeddings, (ProductID3_history))
    x_product_4 = tf.nn.embedding_lookup(product_embeddings, (ProductID4_history))
    x_product_5 = tf.nn.embedding_lookup(product_embeddings, (ProductID5_history))
    x_product_6 = tf.nn.embedding_lookup(product_embeddings, (ProductID6_history))
    x_product_7 = tf.nn.embedding_lookup(product_embeddings, (ProductID7_history))
    x_product_8 = tf.nn.embedding_lookup(product_embeddings, (ProductID8_history))
    x_product_9 = tf.nn.embedding_lookup(product_embeddings, (ProductID9_history))
    x_product_10 = tf.nn.embedding_lookup(product_embeddings, (ProductID10_history))
    x_product_11 = tf.nn.embedding_lookup(product_embeddings, (ProductID11_history))
    x_product_12 = tf.nn.embedding_lookup(product_embeddings, (ProductID12_history))
    x_product_13 = tf.nn.embedding_lookup(product_embeddings, (ProductID13_history))
    x_product_14 = tf.nn.embedding_lookup(product_embeddings, (ProductID14_history))
    x_product_15 = tf.nn.embedding_lookup(product_embeddings, (ProductID15_history))
    x_product_16 = tf.nn.embedding_lookup(product_embeddings, (ProductID16_history))
    x_product_17 = tf.nn.embedding_lookup(product_embeddings, (ProductID17_history))
    x_product_18 = tf.nn.embedding_lookup(product_embeddings, (ProductID18_history))
    x_product_19 = tf.nn.embedding_lookup(product_embeddings, (ProductID19_history))
    x_product_20 = tf.nn.embedding_lookup(product_embeddings, (ProductID20_history))
    
   
    # sequence data
    IsInOrder_history_oh = tf.one_hot(IsInOrder_history, 2)
    order_dow_history_oh = tf.one_hot(order_dow_history, 8)
    order_hour_of_day_history_oh = tf.one_hot(order_hour_of_day_history, 25)
    days_since_prior_order_history_oh = tf.one_hot(days_since_prior_order_history, 31)
    OrderSize_history_oh = tf.one_hot(OrderSize_history, 60)
    IndexInOrder_history_oh = tf.one_hot(OrderSize_history, 60)
    order_number_history_oh = tf.one_hot(order_number_history, 101)
    NumProductsFromDep_history_oh = tf.one_hot(NumProductsFromDep_history, 50)
    
    order_dow_history_scalar = tf.expand_dims(tf.cast(order_dow_history, tf.float32) / 8.0, 2)
    order_hour_of_day_history_scalar = tf.expand_dims(tf.cast(order_hour_of_day_history, tf.float32) / 25.0, 2)
    days_since_prior_order_history_scalar = tf.expand_dims(tf.cast(days_since_prior_order_history, tf.float32) / 31.0, 2)
    OrderSize_history_scalar = tf.expand_dims(tf.cast(OrderSize_history, tf.float32) / 60.0, 2)
    IndexInOrder_history_scalar = tf.expand_dims(tf.cast(IndexInOrder_history, tf.float32) / 60.0, 2)
    order_number_history_scalar = tf.expand_dims(tf.cast(order_number_history, tf.float32) / 100.0, 2)
    NumProductsFromDep_history_scalar = tf.expand_dims(tf.cast(NumProductsFromDep_history, tf.float32) / 50.0, 2)



    x_history = tf.concat([IsInOrder_history_oh,order_dow_history_oh,order_hour_of_day_history_oh,days_since_prior_order_history_oh,\
        OrderSize_history_oh,IndexInOrder_history_oh,NumProductsFromDep_history_oh,order_number_history_oh,order_dow_history_scalar,\
        order_hour_of_day_history_scalar,days_since_prior_order_history_scalar,OrderSize_history_scalar,IndexInOrder_history_scalar,\
        order_number_history_scalar,NumProductsFromDep_history_scalar,x_product_1,x_product_2,x_product_3,\
        x_product_4,x_product_5,x_product_6,x_product_7,x_product_8,x_product_9\
        ,x_product_10,x_product_11,x_product_12,x_product_13,\
        x_product_14,x_product_15,x_product_16,x_product_17,x_product_18,\
        x_product_19,x_product_20,x_product_1a,x_product_2a,x_product_3a,\
        x_product_4a,x_product_5a,x_product_6a,x_product_7a,x_product_8a,x_product_9a\
        ,x_product_10a,x_product_11a,x_product_12a,x_product_13a,\
        x_product_14a,x_product_15a,x_product_16a,x_product_17a,x_product_18a,\
        x_product_19a,x_product_20a,ProductNameEmbedding_history] \
        , axis=2)
    
    x = tf.concat([x_history, x_aisle, x_user], axis=2)
    #h = lstm_layer(x, history_length, lstm_size, scope='lstm1')

    if WaveNet:
        c = wavenet(x, dilations, filter_widths, skip_channels, residual_channels)
        h = c
    else:
        h = lstm_layer(x, history_length, lstm_size, scope='lstm1')
        h = tf.concat([h ], axis=2)
    

    h_final = time_distributed_dense_layer(h, 100 , activation=tf.nn.relu, scope='dense1') 
    #y_hat = tf.squeeze(time_distributed_dense_layer(h_final, 1, activation=tf.nn.sigmoid, scope='dense3'), 2)

    n_components = 1
    params = time_distributed_dense_layer(h_final, n_components*2, scope='dense-2', activation=None)
    ps, mixing_coefs = tf.split(params, 2, axis=2)

    # this is implemented incorrectly, but it still helped...
    mixing_coefs = tf.nn.softmax(mixing_coefs - tf.reduce_min(mixing_coefs, 2, keep_dims=True))
    ps = tf.nn.sigmoid(ps)

    #loss = sequence_log_loss(NextInOrder_history, preds, history_length, N)
    labels = tf.tile(tf.expand_dims(NextInOrder_history, 2), (1, 1, n_components))
    losses = tf.reduce_sum(mixing_coefs*tensor_log_loss(labels, ps), axis=2)
    sequence_mask = tf.cast(tf.sequence_mask(history_length, maxlen=N), tf.float32)
    loss = tf.reduce_sum(losses*sequence_mask) / tf.cast(tf.reduce_sum(history_length), tf.float32)

    
    final_temporal_idx = tf.stack([tf.range(tf.shape(history_length)[0]), history_length - 1], axis=1)
    final_states = tf.gather_nd(h_final, final_temporal_idx)
    #final_predictions = tf.gather_nd(y_hat, final_temporal_idx)
    final_predictions = tf.gather_nd(ps, final_temporal_idx)
    ground_truth = tf.gather_nd(NextInOrder_history, final_temporal_idx)
    
    prediction_tensors = {
        'user_ids': user_id,
        'aisle_ids': master_aisle_id,
        'final_states': final_states,
        'predictions': final_predictions,
        'ground_truth': label, #BUG FIX
        'history_length':history_length
    }
    #preds = y_hat
    preds = ps
        

    global_step = tf.Variable(0, trainable=False)
    learning_rate_var = tf.Variable(0.0, trainable=False)

    if regularization_constant != 0:
        l2_norm = tf.reduce_sum([tf.sqrt(tf.reduce_sum(tf.square(param))) for param in tf.trainable_variables()])
        loss = loss + regularization_constant*l2_norm

    if opt == 'adam':
        optimizer =  tf.train.AdamOptimizer(learning_rate_var)
    elif opt == 'gd':
        optimizer =  tf.train.GradientDescentOptimizer(learning_rate_var)
    elif opt == 'rms':
        optimizer =  tf.train.RMSPropOptimizer(learning_rate_var, decay=0.95, momentum=0.9)
    else:
        assert False, 'optimizer must be adam, gd, or rms'
            
    grads = optimizer.compute_gradients(loss)
    clipped = [(tf.clip_by_value(g, -grad_clip, grad_clip), v_) for g, v_ in grads]

    step = optimizer.apply_gradients(clipped, global_step=global_step)

    print('trainable parameters:')
    print([(var.name, shape(var)) for var in tf.trainable_variables()])

    print('trainable parameter count:')
    print(str(np.sum(np.prod(shape(var)) for var in tf.trainable_variables())))

    saver = tf.train.Saver(max_to_keep=1)
    
    init = tf.global_variables_initializer()
    
session = tf.Session(graph=graph)
print('built graph')

trainable parameters:
[('aisle_embeddings:0', [135, 50]), ('dept_embeddings:0', [22, 10]), ('user_embeddings:0', [207000, 3]), ('product_embeddings:0', [50000, 50]), ('wavenet/x-proj/weights:0', [2457, 128]), ('wavenet/x-proj/biases:0', [128]), ('wavenet/cnn-0/weights:0', [2, 128, 256]), ('wavenet/cnn-0/biases:0', [256]), ('wavenet/cnn-0-proj/weights:0', [128, 192]), ('wavenet/cnn-0-proj/biases:0', [192]), ('wavenet/cnn-1/weights:0', [2, 128, 256]), ('wavenet/cnn-1/biases:0', [256]), ('wavenet/cnn-1-proj/weights:0', [128, 192]), ('wavenet/cnn-1-proj/biases:0', [192]), ('wavenet/cnn-2/weights:0', [2, 128, 256]), ('wavenet/cnn-2/biases:0', [256]), ('wavenet/cnn-2-proj/weights:0', [128, 192]), ('wavenet/cnn-2-proj/biases:0', [192]), ('wavenet/cnn-3/weights:0', [2, 128, 256]), ('wavenet/cnn-3/biases:0', [256]), ('wavenet/cnn-3-proj/weights:0', [128, 192]), ('wavenet/cnn-3-proj/biases:0', [192]), ('wavenet/cnn-4/weights:0', [2, 128, 256]), ('wavenet/cnn-4/biases:0', [256]), ('wavenet/cnn-4-

In [26]:
with session.as_default():
    debug = False

    if warm_start_init_step:
        restore(warm_start_init_step)
        step_number = warm_start_init_step
    else:
        session.run(init)
        step_number = 0

    best_validation_loss, best_validation_tstep = float('inf'), 0
    restarts = 0
    
    #<AUC change>
    
    validation_predictions = np.zeros(shape=[batch_size*log_interval,1],dtype=np.float32)
    validation_aisle = np.zeros(shape=[batch_size*log_interval,1],dtype=np.int32)
    validation_ground_truths = np.zeros(shape=[batch_size*log_interval,1],dtype=np.int32)
    validation_index = 0
    
    #</AUC change>
    
    train_loss_history = deque(maxlen=loss_averaging_window)
    val_loss_history = deque(maxlen=loss_averaging_window)
    #val_loss_history.append(0)
    val_seq_loss_history = deque(maxlen=loss_averaging_window)
    
    while step_number < num_training_steps:

        next_batch =  next(train.next_batch(batch_size))
                    
        # train step
        train_feed_dict = next_batch

        # Use n-2 for training so that n-1 can be used for validation

        if debug:
            print(next_batch[globals()['history_length']])
        train_feed_dict.update({globals()['history_length']:train_feed_dict[globals()['history_length']]-1})
        
        if debug:
            print(train_feed_dict[globals()['history_length']])

        sequence_lengths = train_feed_dict[globals()['history_length']]
        if debug:
            print(sequence_lengths)
        
        for i in range(batch_size):
        
        # Remove the input data for n-1 step for training to avoid leakage

            temp_array = train_feed_dict[globals()['NextInOrder_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['NextInOrder_history']:temp_array})
            
            temp_array = train_feed_dict[globals()['order_number_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['order_number_history']:temp_array})

            temp_array = train_feed_dict[globals()['order_hour_of_day_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['order_hour_of_day_history']:temp_array})

            temp_array = train_feed_dict[globals()['order_dow_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['order_dow_history']:temp_array})

            temp_array = train_feed_dict[globals()['days_since_prior_order_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['days_since_prior_order_history']:temp_array})
                       
            temp_array = train_feed_dict[globals()['IsInOrder_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['IsInOrder_history']:temp_array})

            temp_array = train_feed_dict[globals()['OrderSize_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['OrderSize_history']:temp_array})

            temp_array = train_feed_dict[globals()['IndexInOrder_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['IndexInOrder_history']:temp_array})

            temp_array = train_feed_dict[globals()['NumProductsFromDep_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['NumProductsFromDep_history']:temp_array})

            if True:
                temp_array = train_feed_dict[globals()['ProductID1_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID1_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID2_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID2_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID3_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID3_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID4_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID4_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID5_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID5_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID6_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID6_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID7_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID7_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID8_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID8_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID9_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID9_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID10_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID10_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID11_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID11_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID12_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID12_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID13_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID13_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID14_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID14_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID15_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID15_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID16_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID16_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID17_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID17_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID18_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID18_history_a']:temp_array})      

                temp_array = train_feed_dict[globals()['ProductID19_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID19_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID20_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID20_history_a']:temp_array})

            temp_array = train_feed_dict[globals()['ProductNameEmbedding_history']]
            temp_array[i,sequence_lengths[i]]=np.zeros(shape=[50],dtype=np.float16)
            train_feed_dict.update({globals()['ProductNameEmbedding_history']:temp_array})
            
            temp_array = train_feed_dict[globals()['ProductID1_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID1_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID2_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID2_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID3_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID3_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID4_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID4_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID5_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID5_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID6_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID6_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID7_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID7_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID8_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID8_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID9_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID9_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID10_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID10_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID11_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID11_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID12_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID12_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID13_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID13_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID14_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID14_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID15_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID15_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID16_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID16_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID17_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID17_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID18_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID18_history']:temp_array})      

            temp_array = train_feed_dict[globals()['ProductID19_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID19_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID20_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID20_history']:temp_array})

            
            
        train_feed_dict.update({learning_rate_var: learning_rate})
        train_feed_dict.update({keep_prob: keep_prob_scalar})
        train_feed_dict.update({is_training: True})
                
        debug_dict = {tensor_name: [] for tensor_name in prediction_tensors} #changed
        tensor_names, tf_tensors = zip(*prediction_tensors.items()) #changed

        train_loss,_,np_tensors = session.run(   #changed
            fetches=[loss,step,tf_tensors], #changed
            feed_dict=train_feed_dict
        )
        
        for tensor_name, tensor in zip(tensor_names, np_tensors): #changed
            debug_dict[tensor_name].append(tensor)#changed 
        
        if debug:
            print(debug_dict['history_length'])#changed
        
        train_loss_history.append(train_loss)
        
        #BEGIN
        prediction_dict = {tensor_name: [] for tensor_name in prediction_tensors}

        # test evaluation
        
        val_feed_dict = next_batch
        val_feed_dict.update({learning_rate_var: learning_rate})
        val_feed_dict.update({keep_prob: keep_prob_scalar})
        val_feed_dict.update({is_training: False})

        if debug:
            print(val_feed_dict[globals()['history_length']]+1)
        
        tensor_names, tf_tensors = zip(*prediction_tensors.items())
        val_seq_loss,np_tensors = session.run(
            fetches=[loss,tf_tensors],
            feed_dict=val_feed_dict
        )

        for tensor_name, tensor in zip(tensor_names, np_tensors):
            prediction_dict[tensor_name].append(tensor)
        #print(prediction_dict['ground_truth'])
        #print(prediction_dict['predictions'])
        if(np.sum(prediction_dict['ground_truth'][0])>0):
            val_loss = log_loss(prediction_dict['ground_truth'][0],prediction_dict['predictions'][0])
        #END
            val_loss_history.append(val_loss)
        val_seq_loss_history.append(val_seq_loss)
        #<AUC change>
        #print(validation_index,(validation_index+batch_size),(log_interval*batch_size))
        validation_predictions[validation_index:(validation_index+batch_size)] = np.reshape(prediction_dict['predictions'][0],(batch_size,1))
        validation_aisle[validation_index:(validation_index+batch_size)] = np.reshape(prediction_dict['aisle_ids'][0],(batch_size,1))
        validation_ground_truths[validation_index:(validation_index+batch_size)] = np.reshape(prediction_dict['ground_truth'][0],(batch_size,1))
        validation_index += batch_size
        #</AUC change>
        if debug:
            #print(train_loss, val_loss)
            #print(prediction_dict['ground_truth'])
            print(prediction_dict['history_length'])
            break
        
        if step_number % (log_interval-1) == 0: #<AUC CHange>
            avg_train_loss = sum(train_loss_history)/len(train_loss_history)
            denominator = len(val_loss_history)
            if denominator ==0:
                denominator=1
            avg_val_loss = sum(val_loss_history)/denominator
            avg_val_seq_loss = sum(val_seq_loss_history)/len(val_seq_loss_history)
            
             #<AUC change>  
            if step_number>0:
                avg_val_loss = -findAverageAUC(validation_predictions,validation_aisle,validation_ground_truths)
                validation_predictions = np.zeros(shape=[batch_size*log_interval,1],dtype=np.float32)
                validation_aisle = np.zeros(shape=[batch_size*log_interval,1],dtype=np.int32)
                validation_ground_truths = np.zeros(shape=[batch_size*log_interval,1],dtype=np.int32)
                validation_index = 0
            else:
                avg_val_loss = 0
            #</AUC change>
            
            metric_log = (
                "[[step {:>8}]]     "
                "[[train]]     loss: {:<12}     "
                "[[val]]     loss: {:<12}     "
                "[[val-seq]]     loss: {:<12}     "
            ).format(step_number, round(avg_train_loss, 8), round(-avg_val_loss, 8), round(avg_val_seq_loss, 8)) #<AUC Change>
            print(metric_log)
            
            if avg_val_loss < best_validation_loss:
                best_validation_loss = avg_val_loss
                best_validation_tstep = step_number
                if step_number > min_steps_to_checkpoint:
                    model_path = os.path.join(checkpoint_dir, 'model{}.ckpt'.format(best_validation_tstep))
                    saver.save(session, model_path, global_step=best_validation_tstep) 
                    
            if step_number - best_validation_tstep > early_stopping_steps:

                if num_restarts is None or restarts >= num_restarts:
                    print('best validation loss of {} at training step {}'.format(
                        best_validation_loss, best_validation_tstep))
                    print('early stopping - ending training.')
                    break

                if restarts < num_restarts:
                    model_path = os.path.join(checkpoint_dir, 'model{}.ckpt-{}'.format(best_validation_tstep,best_validation_tstep))
                    saver.restore(session, model_path)
                    print('halving learning rate')
                    learning_rate /= 2.0
                    early_stopping_steps /= 2
                    step_number = best_validation_tstep
                    restarts += 1

        step_number += 1

    if step_number <= min_steps_to_checkpoint:
        best_validation_tstep = step_number
        model_path = os.path.join(checkpoint_dir, 'model{}.ckpt'.format(best_validation_tstep))
        saver.save(session, model_path, global_step=best_validation_tstep) 
                    
    print('num_training_steps reached - ending training')

[[step        0]]     [[train]]     loss: 0.61200756       [[val]]     loss: 0                [[val-seq]]     loss: 0.397838         
[[step      999]]     [[train]]     loss: 0.16791001       [[val]]     loss: 0.50176212       [[val-seq]]     loss: 0.16738042       


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


[[step     1998]]     [[train]]     loss: 0.1661299        [[val]]     loss: 0.50443552       [[val-seq]]     loss: 0.16611269       
[[step     2997]]     [[train]]     loss: 0.16488369       [[val]]     loss: 0.49103145       [[val-seq]]     loss: 0.16486529       
[[step     3996]]     [[train]]     loss: 0.16505543       [[val]]     loss: 0.50392556       [[val-seq]]     loss: 0.16503042       


KeyboardInterrupt: 

In [23]:
best_validation_loss, best_validation_tstep

(0.41730564314492313, 31000)

In [24]:
best_validation_tstep = 31000

In [25]:
model_path = os.path.join(checkpoint_dir, 'model{}.ckpt'.format(best_validation_tstep))
model_path
saver.save(session, model_path, global_step=best_validation_tstep) 

'/userdata/projects/instakart/np/checkpoints/model31000.ckpt-31000'

In [26]:
val = dataset(path_out,"")

In [27]:
model_path='/userdata/projects/instakart/np/checkpoints/model31000.ckpt-31000'
saver.restore(session, model_path)

INFO:tensorflow:Restoring parameters from /userdata/projects/instakart/np/checkpoints/model31000.ckpt-31000


In [28]:
prediction_dict = {tensor_name: [] for tensor_name in prediction_tensors}

num_val_batches = int(val.arrays[0].shape[0]/batch_size)
num_val_batches = int(5000000/batch_size)
print(num_val_batches)

39062


In [29]:
tic()
for i in range(num_val_batches):
    val_dict_list =  next(val.next_batch(batch_size))
        
    # test evaluation
    val_feed_dict = val_dict_list
    val_feed_dict.update({learning_rate_var: learning_rate})
    val_feed_dict.update({keep_prob: 1.0})
    val_feed_dict.update({is_training: False})

    tensor_names, tf_tensors = zip(*prediction_tensors.items())
    np_tensors = session.run(
        fetches=tf_tensors,
        feed_dict=val_feed_dict
    )
    for tensor_name, tensor in zip(tensor_names, np_tensors):
        prediction_dict[tensor_name].append(tensor)
toc()

took me 710.586 seconds to do this..


In [30]:
for tensor_name, tensor in prediction_dict.items():
    np_tensor = np.concatenate(tensor, 0)
    save_file = os.path.join(prediction_dir, '{}.npy'.format(tensor_name))
    print('saving {} with shape {} to {}'.format(tensor_name, np_tensor.shape, save_file))
    np.save(save_file, np_tensor)

saving user_ids with shape (4999936,) to /userdata/projects/instakart/np/predictions/user_ids.npy
saving aisle_ids with shape (4999936,) to /userdata/projects/instakart/np/predictions/aisle_ids.npy
saving final_states with shape (4999936, 10) to /userdata/projects/instakart/np/predictions/final_states.npy
saving predictions with shape (4999936, 1) to /userdata/projects/instakart/np/predictions/predictions.npy
saving ground_truth with shape (4999936,) to /userdata/projects/instakart/np/predictions/ground_truth.npy
saving history_length with shape (4999936,) to /userdata/projects/instakart/np/predictions/history_length.npy


In [31]:
def findLift(predics,actual, user_ids , a_ids ,verbosity=0):
    
    aisles = np.unique(a_ids)
    users = np.unique(user_ids)
    cum_lift = 0.0
    cnt = 0

    for n in range(len(aisles)):

        predics_aisle = predics[np.where(a_ids==aisles[n])]
        actual_aisle = actual[np.where(a_ids==aisles[n])]
        users_aisle = user_ids[np.where(a_ids==aisles[n])]

        i = np.where(predics_aisle>np.percentile(predics_aisle,90))
        j = np.where(predics_aisle>0)
        num_i = len(i[0])*1.0
        num_j = len(j[0])*1.0
        num_users = users.shape[0]

        up = (np.sum(actual_aisle[i[0]])*1.0)/num_i
        down = (np.sum(actual_aisle[j[0]])*1.0)/num_j
        lift = round(up/down,3)
        if lift>0:
            cum_lift += lift
            cnt += 1

        if verbosity==1:
            print(lift , round(up,2),round(down,2), round(num_i/num_j,3), round((num_j*100.0)/num_users,2))

    return round(cum_lift/cnt,2)

In [33]:
predics = np.load(prediction_dir+'/predictions.npy')
actual = np.load(prediction_dir+'/ground_truth.npy')
a_ids = np.load(prediction_dir+'/aisle_ids.npy')
u_ids = np.load(prediction_dir+'/user_ids.npy')
print(predics.shape,actual.shape, a_ids.shape, u_ids.shape)

(4999936, 1) (4999936,) (4999936,) (4999936,)


In [34]:
print(findLift(predics,actual,u_ids, a_ids,True))

3.789 0.46 0.12 0.1 8.87
4.55 0.5 0.11 0.1 13.45
2.647 0.44 0.17 0.1 27.23
3.167 0.45 0.14 0.1 23.15
5.697 0.46 0.08 0.1 14.01
5.551 0.51 0.09 0.1 7.56
4.803 0.48 0.1 0.1 5.98
5.178 0.49 0.1 0.1 5.65
3.506 0.43 0.12 0.1 29.19
7.86 0.49 0.06 0.1 2.57
6.571 0.54 0.08 0.1 5.39
4.719 0.47 0.1 0.1 5.13
3.302 0.46 0.14 0.1 10.86
3.038 0.45 0.15 0.1 11.45
4.905 0.46 0.09 0.1 3.9
2.834 0.44 0.16 0.1 32.22
3.773 0.42 0.11 0.1 36.79
4.286 0.48 0.11 0.1 2.68
4.111 0.43 0.1 0.1 36.63
5.199 0.48 0.09 0.1 11.66
2.066 0.47 0.23 0.1 56.56
5.74 0.44 0.08 0.1 6.14
3.39 0.45 0.13 0.1 19.0
1.482 0.6 0.4 0.1 74.64
5.19 0.44 0.09 0.1 10.9
3.009 0.45 0.15 0.1 19.88
3.094 0.41 0.13 0.1 3.42
3.419 0.41 0.12 0.1 2.88
5.831 0.46 0.08 0.1 14.51
4.745 0.47 0.1 0.1 13.19
2.498 0.49 0.2 0.1 35.94
2.509 0.52 0.21 0.1 19.7
5.496 0.51 0.09 0.1 2.19
4.024 0.47 0.12 0.1 11.03
3.526 0.45 0.13 0.1 16.43
3.549 0.44 0.12 0.1 33.68
2.693 0.46 0.17 0.1 38.04
2.654 0.47 0.18 0.1 24.89
5.203 0.44 0.08 0.1 5.41
3.237 0.46 0.14 0.

# Log

Things to try:

1. Batch Normalization
2. 50, 300 User Embeddings
3. Department Embeddings
4. Product Level Model Stacked with Department Level Model
5. wavenet
6. dense layer